# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 4 2022 12:41PM,247857,10,MSP214465,"Methapharm, Inc.",Released
1,Oct 4 2022 12:41PM,247857,10,MSP214466,"Methapharm, Inc.",Released
2,Oct 4 2022 12:41PM,247857,10,MSP214468,"Methapharm, Inc.",Released
3,Oct 4 2022 12:41PM,247857,10,MSP214485,"Methapharm, Inc.",Released
4,Oct 4 2022 12:41PM,247857,10,MSP214470,"Methapharm, Inc.",Released
5,Oct 4 2022 12:41PM,247857,10,MSP214473,"Methapharm, Inc.",Released
6,Oct 4 2022 12:41PM,247857,10,MSP214474,"Methapharm, Inc.",Released
7,Oct 4 2022 12:41PM,247857,10,MSP214476,"Methapharm, Inc.",Released
8,Oct 4 2022 12:41PM,247857,10,MSP214477,"Methapharm, Inc.",Released
9,Oct 4 2022 12:41PM,247857,10,MSP214479,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
22,247854,Released,2
23,247855,Released,2
24,247856,Released,45
25,247857,Released,40
26,247858,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
247854,NaN,2.0
247855,NaN,2.0
247856,NaN,45.0
247857,NaN,40.0
247858,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
247764,0.0,12.0
247794,0.0,1.0
247796,3.0,5.0
247811,4.0,26.0
247815,12.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
247764,0,12
247794,0,1
247796,3,5
247811,4,26
247815,12,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,247764,0,12
1,247794,0,1
2,247796,3,5
3,247811,4,26
4,247815,12,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,247764,,12
1,247794,,1
2,247796,3,5
3,247811,4,26
4,247815,12,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 4 2022 12:41PM,247857,10,"Methapharm, Inc."
40,Oct 4 2022 12:41PM,247856,10,"Methapharm, Inc."
85,Oct 4 2022 12:40PM,247858,10,"Methapharm, Inc."
88,Oct 4 2022 12:36PM,247855,19,ACG North America LLC
90,Oct 4 2022 12:33PM,247854,19,Paragon BioTeck
92,Oct 4 2022 12:32PM,247853,10,MedStone Pharma LLC
98,Oct 4 2022 12:31PM,247852,10,Emerginnova
100,Oct 4 2022 12:27PM,247851,10,"CLINUVEL, Inc."
101,Oct 4 2022 11:53AM,247848,20,"Exact-Rx, Inc."
102,Oct 4 2022 11:36AM,247840,20,"Exact-Rx, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Oct 4 2022 12:41PM,247857,10,"Methapharm, Inc.",,40
1,Oct 4 2022 12:41PM,247856,10,"Methapharm, Inc.",,45
2,Oct 4 2022 12:40PM,247858,10,"Methapharm, Inc.",,3
3,Oct 4 2022 12:36PM,247855,19,ACG North America LLC,,2
4,Oct 4 2022 12:33PM,247854,19,Paragon BioTeck,,2
5,Oct 4 2022 12:32PM,247853,10,MedStone Pharma LLC,,6
6,Oct 4 2022 12:31PM,247852,10,Emerginnova,,2
7,Oct 4 2022 12:27PM,247851,10,"CLINUVEL, Inc.",,1
8,Oct 4 2022 11:53AM,247848,20,"Exact-Rx, Inc.",,1
9,Oct 4 2022 11:36AM,247840,20,"Exact-Rx, Inc.",5,22


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 4 2022 12:41PM,247857,10,"Methapharm, Inc.",40,
1,Oct 4 2022 12:41PM,247856,10,"Methapharm, Inc.",45,
2,Oct 4 2022 12:40PM,247858,10,"Methapharm, Inc.",3,
3,Oct 4 2022 12:36PM,247855,19,ACG North America LLC,2,
4,Oct 4 2022 12:33PM,247854,19,Paragon BioTeck,2,
5,Oct 4 2022 12:32PM,247853,10,MedStone Pharma LLC,6,
6,Oct 4 2022 12:31PM,247852,10,Emerginnova,2,
7,Oct 4 2022 12:27PM,247851,10,"CLINUVEL, Inc.",1,
8,Oct 4 2022 11:53AM,247848,20,"Exact-Rx, Inc.",1,
9,Oct 4 2022 11:36AM,247840,20,"Exact-Rx, Inc.",22,5


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 4 2022 12:41PM,247857,10,"Methapharm, Inc.",40,
1,Oct 4 2022 12:41PM,247856,10,"Methapharm, Inc.",45,
2,Oct 4 2022 12:40PM,247858,10,"Methapharm, Inc.",3,
3,Oct 4 2022 12:36PM,247855,19,ACG North America LLC,2,
4,Oct 4 2022 12:33PM,247854,19,Paragon BioTeck,2,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Oct 4 2022 12:41PM,247857,10,"Methapharm, Inc.",40.0,NaN
1,Oct 4 2022 12:41PM,247856,10,"Methapharm, Inc.",45.0,NaN
2,Oct 4 2022 12:40PM,247858,10,"Methapharm, Inc.",3.0,NaN
3,Oct 4 2022 12:36PM,247855,19,ACG North America LLC,2.0,NaN
4,Oct 4 2022 12:33PM,247854,19,Paragon BioTeck,2.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 4 2022 12:41PM,247857,10,"Methapharm, Inc.",40.0,0.0
1,Oct 4 2022 12:41PM,247856,10,"Methapharm, Inc.",45.0,0.0
2,Oct 4 2022 12:40PM,247858,10,"Methapharm, Inc.",3.0,0.0
3,Oct 4 2022 12:36PM,247855,19,ACG North America LLC,2.0,0.0
4,Oct 4 2022 12:33PM,247854,19,Paragon BioTeck,2.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3469601,161.0,19.0
16,247796,5.0,3.0
18,247839,0.0,1.0
19,495709,4.0,0.0
20,743517,24.0,5.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3469601,161.0,19.0
1,16,247796,5.0,3.0
2,18,247839,0.0,1.0
3,19,495709,4.0,0.0
4,20,743517,24.0,5.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,161.0,19.0
1,16,5.0,3.0
2,18,0.0,1.0
3,19,4.0,0.0
4,20,24.0,5.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,161.0
1,16,Released,5.0
2,18,Released,0.0
3,19,Released,4.0
4,20,Released,24.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,18,19,20
Status,,,,,
Executing,19.0,3.0,1.0,0.0,5.0
Released,161.0,5.0,0.0,4.0,24.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,18,19,20
0,Executing,19.0,3.0,1.0,0.0,5.0
1,Released,161.0,5.0,0.0,4.0,24.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,18,19,20
0,Executing,19.0,3.0,1.0,0.0,5.0
1,Released,161.0,5.0,0.0,4.0,24.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()